In [78]:
import pandas as pd

In [79]:
feature_df = pd.read_csv('features.csv', skiprows=[1,2,3], low_memory=False)

In [80]:
feature_df = feature_df.rename(columns={'feature':'track_id'})

In [81]:
core_df = pd.read_csv('tracks.csv', skiprows=1, index_col=0, engine='python', error_bad_lines=False)

Skipping line 53396: unexpected end of data


In [82]:
tracks = core_df[ ['producer', 'location', 'composer','type','id','genre_top'] ]
tracks = tracks.drop('track_id', axis=0)

In [83]:
tracks = tracks.reset_index()
tracks = tracks.rename(columns={'index':'track_id'})

In [84]:
tracks['track_id'] = tracks['track_id'].astype(int)

In [85]:
tracks = tracks.dropna(subset=['genre_top'])

In [86]:
merged = pd.merge(tracks, feature_df, on='track_id', how='left')

In [87]:
merged = merged.drop(columns={'track_id', 'id'})

In [88]:
from sklearn.model_selection import train_test_split

X = merged.drop(columns='genre_top', axis=1)
y = merged['genre_top']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((22740, 522), (5686, 522), (22740,), (5686,))

In [89]:
X_train.head()

,producer,location,composer,type,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
5070,NaN,Stuttgart - Germany,NaN,Album,0.228840,-0.512770,-0.777282,-0.490554,-0.377102,-0.038641,...,0.087990,0.015597,0.016914,5.482908,0.164062,0.04251,0.040039,0.0,1.681478,0.019171
27666,NaN,"New York, NY",NaN,Album,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2859,Terre T,"Atlanta, GA",NaN,Radio Program,-0.458388,0.279253,1.447103,1.007605,0.182779,0.190900,...,0.056364,0.012418,0.012510,0.445951,0.300781,0.09109,0.084961,0.0,0.628830,0.037491
25798,Joe Belock,"Brooklyn, NY",NaN,Album,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17578,NaN,NaN,NaN,Album,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
import category_encoders as ce


dummy_encoder = ce.OneHotEncoder(use_cat_names=True)
dummy_encoder = dummy_encoder.fit(X_train['type'])

dummy_train = dummy_encoder.transform(X_train.loc[:,'type'])
dummy_test = dummy_encoder.transform(X_test.loc[:,'type'])

X_train = pd.concat([X_train, dummy_train], axis=1)
X_test = pd.concat([X_test, dummy_test], axis=1)

In [91]:
hash_features = ['producer', 'location', 'composer']

for feature in hash_features:
    hasher = ce.HashingEncoder(n_components=10, cols=feature)

    hasher = hasher.fit(X_train[feature])

    hasher_train = hasher.transform(X_train.loc[:,feature])
    hasher_test = hasher.transform(X_test.loc[:,feature])

    X_train = pd.concat([X_train, hasher_train], axis=1)
    X_test = pd.concat([X_test, hasher_test], axis=1)

In [92]:
X_train.shape, X_test.shape

((22740, 557), (5686, 557))

In [93]:
X_train = X_train.drop(columns={'producer', 'location','composer','type'}, axis=1)

In [94]:
X_test = X_test.drop(columns={'producer', 'location','composer','type'}, axis=1)

In [95]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
X_train = pd.DataFrame( imputer.fit_transform(X_train) )
X_test = pd.DataFrame( imputer.transform(X_test) )

In [96]:
X_train.isnull().sum().sum()

0

In [97]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='lbfgs', max_iter=2000, multi_class='auto')
log_reg = log_reg.fit(X_train, y_train)

score = log_reg.score(X_train, y_train)
print('Train Accuracy Score:', score)
score = log_reg.score(X_test, y_test)
print('Train Accuracy Score:', score)

Train Accuracy Score: 0.38007915567282324
Train Accuracy Score: 0.3760112557157932


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
